In [1]:
pip install openprompt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv

# 初始化字典
trec_dict = {}
q_dict = {}

# 从trec.txt中读取数据
with open('trec.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        query_content = row[1]  # 第二列是查询内容
        trec_dict[query_id] = query_content

# 从lmdir.txt中读取数据
with open('lmdir.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        query_id = row[0]  # 第一列是查询id
        doc_content = row[6]  # 第七列是文档内容
        # 计算当前查询id的文档数量
        doc_number = len(q_dict.get(query_id, [])) + 1
        # 在文档内容前添加标识
        doc_content = f'Document: {doc_content}'
        # 将查询id和带标识的文档内容添加到字典中
        key = query_id + '\t' + trec_dict.get(query_id, '')
        if key in q_dict:
            q_dict[key] = q_dict[key] + ' ' + doc_content
        else:
            q_dict[key] = doc_content


# 打印出第一个查询id及其对应的文档内容，以验证代码是否正确
for query_id in list(q_dict.keys())[:1]:
    print("Query :", query_id)
    print("Document Contents:", q_dict[query_id])


Query : 264014	how long is life cycle of flea
Document Contents: Document:  _NUM_  Cancel. A flea can live up to a year, but its general lifespan depends on its living conditions, such as the availability of hosts. Find out how long a flea's life cycle can last with tips from a pet industry specialist in this free video on fleas and pest control.Part of the Video Series: Flea Control.ancel. A flea can live up to a year, but its general lifespan depends on its living conditions, such as the availability of hosts. Find out how long a flea's life cycle can last with tips from a pet industry specialist in this free video on fleas and pest control. Part of the Video Series: Flea Control. Document: The cat flea's primary host is the domestic cat, but it is also the primary flea infesting dogs in most of the world. The cat flea can also maintain its life cycle on other carnivores and on omnivores. Humans can be bitten, though a long-term population of cat fleas cannot be sustained and infest 

In [2]:
import json

def load_dict_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# 使用示例
q_dict = load_dict_from_file('q_dict.txt')

In [3]:
import itertools
from openprompt.data_utils import InputExample

input_examples = []

# 使用itertools.combinations来获取所有的两两组合，不会重复
for querya_id_content, queryb_id_content in itertools.combinations(q_dict.keys(), 2):
    # 分割查询ID和查询内容
    querya_id, querya = querya_id_content.split('\t', 1)
    documentsa = q_dict[querya_id_content]
    
    queryb_id, queryb = queryb_id_content.split('\t', 1)
    documentsb = q_dict[queryb_id_content]
    
    t=querya_id+':'+queryb_id
    
    # 构建InputExample
    input_example = InputExample(
        guid= len(input_examples),
        label= None,
        meta= {
            "queryaid": querya_id,
            "querya": querya,
            "documentsa": documentsa,
            "querybid": queryb_id,
            "queryb": queryb,
            "documentsb": documentsb
        },
        text_a= "",
        text_b= "",
        tgt_text= t
    )
    
    # 将InputExample添加到列表中
    input_examples.append(input_example)


D:\pd\Anaconda3\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [4]:
input_examples[0]

{
  "guid": 0,
  "label": null,
  "meta": {
    "documentsa": [
      " This video offers advice from a pet industry expert on how to manage fleas and how long their life cycle can last, which is determined by their environment.",
      " The cat flea is a common flea that infests cats and dogs, as well as other carnivores and omnivores. Although humans can be bitten, they cannot sustain a long-term population of cat fleas. However, if a female flea is allowed to feed on a human for a certain number of hours, it can lay viable eggs.",
      " Fleas have a life cycle that can last from a few days to a year, and is affected by weather conditions. Female fleas can lay up to 20 eggs per day, which take two to six days to hatch and make up one third of the flea population.",
      " Document 4 explains the flea life cycle, which begins with adult fleas laying eggs on a host animal, which then fall off and hatch into first stage flea larvae.",
      " Flea larvae are small, white, and legles

In [34]:
# class MyInput(object):
#     def __init__(self, guid, querya, documentsa, queryb, documentsb, label=None, meta={}):
#         self.guid = guid
#         self.querya = querya
#         self.documentsa = documentsa
#         self.queryb = queryb
#         self.documentsb = documentsb
#         self.label = label
#         self.meta = meta

#     def __repr__(self):
#         return str(self.__dict__)


In [5]:
# query_pairs = [(querya_id, queryb_id) for querya_id in q_dict for queryb_id in q_dict if querya_id > queryb_id]

In [6]:

# examples = []
# for i, (querya_id, queryb_id) in enumerate(query_pairs):
#     querya = querya_id
#     documentsa = q_dict[querya_id]
#     queryb = queryb_id
#     documentsb = q_dict[queryb_id]
#     examples.append(MyInput(guid=i, querya=querya, documentsa=documentsa, queryb=queryb, documentsb=documentsb))


In [5]:
for example in input_examples:
    # 对'documentsa'列表中的文档编上号并拼接
    documentsa = example.meta['documentsa']
    documentsa = [f'Document {i+1}: {doc}' for i, doc in enumerate(documentsa)]
    example.meta['documentsa'] = ' '.join(documentsa)

    # 对'documentsb'列表中的文档编上号并拼接
    documentsb = example.meta['documentsb']
    documentsb = [f'Document {i+1}: {doc}' for i, doc in enumerate(documentsb)]
    example.meta['documentsb'] = ' '.join(documentsb)

In [6]:
input_examples[0]

{
  "guid": 0,
  "label": null,
  "meta": {
    "documentsa": "Document 1:  This video offers advice from a pet industry expert on how to manage fleas and how long their life cycle can last, which is determined by their environment. Document 2:  The cat flea is a common flea that infests cats and dogs, as well as other carnivores and omnivores. Although humans can be bitten, they cannot sustain a long-term population of cat fleas. However, if a female flea is allowed to feed on a human for a certain number of hours, it can lay viable eggs. Document 3:  Fleas have a life cycle that can last from a few days to a year, and is affected by weather conditions. Female fleas can lay up to 20 eggs per day, which take two to six days to hatch and make up one third of the flea population. Document 4:  Document 4 explains the flea life cycle, which begins with adult fleas laying eggs on a host animal, which then fall off and hatch into first stage flea larvae. Document 5:  Flea larvae are small, w

In [7]:
from openprompt.data_utils import InputExample
classes = [ # There are three docs
    "specific",
    "generic"
]
dataset = input_examples

In [8]:
# from transformers import RobertaTokenizer, RobertaModel
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaModel.from_pretrained('roberta-base')

In [9]:
from openprompt.plms import load_plm
#plm, tokenizer, model_config, WrapperClass = load_plm("t5", "google/flan-t5-base")
plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "gpt2")


Using pad_token, but it is not set yet.


In [19]:
from openprompt.prompts import ManualTemplate

# placeholder_mapping = {
#     '$Querya': 'Querya',
#     '$Documentsa': 'Documentsa',
#     '$Queryb': 'Queryb',
#     '$Documentsb': 'Documentsb',
# }

template_text = 'Given the following two queries and their respective top five documents: the first query is "{"meta": "querya"}" with its top five search results as "{"meta": "documentsa", "shortenable": True}", and the second query is "{"meta": "queryb"}" with its top five search results as "{"meta": "documentsb", "shortenable": True}", it can be inferred that the second query is more {"mask"}.'
#template_text = 'Query "{"meta": "querya"}" , the second query "{"meta": "queryb"}" the second query is more {"mask"}'


promptTemplate = ManualTemplate(
    text = template_text,
    tokenizer = tokenizer,
    
)

In [20]:
from openprompt.prompts import ManualVerbalizer
promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "specific":["specific","precise","exact"],
        "generic":["generic","general","common","universal","useless"]
    },
    tokenizer = tokenizer,
)

In [21]:
# model.wrapper

In [22]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

In [23]:
from openprompt import PromptDataLoader
from openprompt.plms.utils import TokenizerWrapper
data_loader = PromptDataLoader(
    dataset = dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
)

tokenizing: 903it [00:03, 240.70it/s]


In [24]:
import numpy as np

def softmax(logits):
    e = np.exp(logits)
    return e / np.sum(e)

logits = np.array([-1.5303, -2.4777])
probabilities = softmax(logits)

print(probabilities)


[0.720592 0.279408]


In [25]:
from collections import defaultdict
import torch
# 初始化一个字典来存储每个query的总分数
scores = defaultdict(int)

# 对于数据加载器中的每个批次
for batch in data_loader:
    # 获取模型的预测结果
    with torch.no_grad():
        logits = promptModel(batch)
        preds = torch.argmax(logits, dim = -1)
        probs = torch.softmax(logits, dim=-1)
        
        
    for i, prob in enumerate(probs):
        # 获取query的id
        query_id = batch.tgt_text[i].split(':')[1]
        
        print(prob[classes.index('specific')],prob[classes.index('generic')])

        # 根据模型的预测概率给query加分
        scores[query_id] += prob[classes.index('specific')]

#     # 对于批次中的每个预测结果
#     for i, pred in enumerate(preds):
#         # 获取query的id
#         query_id = batch.tgt_text[i].split(':')[1]

#         # 如果预测结果是'specific'，则给这个query加2分
#         # 如果预测结果是'generic'，则给这个query加1分
#         # 这是一个假设的分数分配方式，实际的分数分配方式可能需要根据具体情况进行调整
#         print(classes[pred])
#         if classes[pred] == 'specific':
#             scores[query_id] += 2
#         else:
#             scores[query_id] += 0

# 最后，我们得到了一个字典，其中每个query的id对应其总分数
# 我们可以根据这个字典来进行排序
sorted_queries = sorted(scores.items(), key=lambda item: item[1], reverse=True)


tensor(0.8516) tensor(0.1484)
tensor(0.8212) tensor(0.1788)
tensor(0.6950) tensor(0.3050)
tensor(0.6870) tensor(0.3130)
tensor(0.7773) tensor(0.2227)
tensor(0.7072) tensor(0.2928)
tensor(0.8975) tensor(0.1025)
tensor(0.6453) tensor(0.3547)
tensor(0.8408) tensor(0.1592)
tensor(0.7197) tensor(0.2803)
tensor(0.8655) tensor(0.1345)
tensor(0.7751) tensor(0.2249)
tensor(0.7812) tensor(0.2188)
tensor(0.7079) tensor(0.2921)
tensor(0.6963) tensor(0.3037)
tensor(0.7784) tensor(0.2216)
tensor(0.7961) tensor(0.2039)
tensor(0.8116) tensor(0.1884)
tensor(0.7831) tensor(0.2169)
tensor(0.6643) tensor(0.3357)
tensor(0.7721) tensor(0.2279)
tensor(0.8850) tensor(0.1150)
tensor(0.7343) tensor(0.2657)
tensor(0.8198) tensor(0.1802)
tensor(0.7965) tensor(0.2035)
tensor(0.6867) tensor(0.3133)
tensor(0.8744) tensor(0.1256)
tensor(0.6406) tensor(0.3594)
tensor(0.7875) tensor(0.2125)
tensor(0.8089) tensor(0.1911)
tensor(0.8068) tensor(0.1932)
tensor(0.6306) tensor(0.3694)
tensor(0.6547) tensor(0.3453)
tensor(0.8

tensor(0.7210) tensor(0.2790)
tensor(0.7225) tensor(0.2775)
tensor(0.8348) tensor(0.1652)
tensor(0.7492) tensor(0.2508)
tensor(0.7060) tensor(0.2940)
tensor(0.6845) tensor(0.3155)
tensor(0.6222) tensor(0.3778)
tensor(0.7426) tensor(0.2574)
tensor(0.7225) tensor(0.2775)
tensor(0.7681) tensor(0.2319)
tensor(0.7298) tensor(0.2702)
tensor(0.5512) tensor(0.4488)
tensor(0.7504) tensor(0.2496)
tensor(0.7596) tensor(0.2404)
tensor(0.6903) tensor(0.3097)
tensor(0.7683) tensor(0.2317)
tensor(0.7161) tensor(0.2839)
tensor(0.6980) tensor(0.3020)
tensor(0.8147) tensor(0.1853)
tensor(0.5941) tensor(0.4059)
tensor(0.7332) tensor(0.2668)
tensor(0.7046) tensor(0.2954)
tensor(0.7378) tensor(0.2622)
tensor(0.5357) tensor(0.4643)
tensor(0.5412) tensor(0.4588)
tensor(0.8232) tensor(0.1768)
tensor(0.6903) tensor(0.3097)
tensor(0.7193) tensor(0.2807)
tensor(0.5916) tensor(0.4084)
tensor(0.7559) tensor(0.2441)
tensor(0.8481) tensor(0.1519)
tensor(0.7111) tensor(0.2889)
tensor(0.7886) tensor(0.2114)
tensor(0.6

tensor(0.7417) tensor(0.2583)
tensor(0.8227) tensor(0.1773)
tensor(0.7383) tensor(0.2617)
tensor(0.6646) tensor(0.3354)
tensor(0.7904) tensor(0.2096)
tensor(0.8440) tensor(0.1560)
tensor(0.7629) tensor(0.2371)
tensor(0.5912) tensor(0.4088)
tensor(0.7521) tensor(0.2479)
tensor(0.8913) tensor(0.1087)
tensor(0.7527) tensor(0.2473)
tensor(0.8579) tensor(0.1421)
tensor(0.8196) tensor(0.1804)
tensor(0.8072) tensor(0.1928)
tensor(0.8540) tensor(0.1460)
tensor(0.6986) tensor(0.3014)
tensor(0.8092) tensor(0.1908)
tensor(0.8247) tensor(0.1753)
tensor(0.8308) tensor(0.1692)
tensor(0.5367) tensor(0.4633)
tensor(0.6039) tensor(0.3961)
tensor(0.8607) tensor(0.1393)
tensor(0.7632) tensor(0.2368)
tensor(0.8323) tensor(0.1677)
tensor(0.7119) tensor(0.2881)
tensor(0.7426) tensor(0.2574)
tensor(0.8635) tensor(0.1365)
tensor(0.8781) tensor(0.1219)
tensor(0.7908) tensor(0.2092)
tensor(0.7231) tensor(0.2769)
tensor(0.8383) tensor(0.1617)
tensor(0.8101) tensor(0.1899)
tensor(0.5989) tensor(0.4011)
tensor(0.7

tensor(0.8331) tensor(0.1669)
tensor(0.7896) tensor(0.2104)
tensor(0.7232) tensor(0.2768)
tensor(0.7942) tensor(0.2058)
tensor(0.6339) tensor(0.3661)
tensor(0.7737) tensor(0.2263)
tensor(0.8711) tensor(0.1289)
tensor(0.8326) tensor(0.1674)
tensor(0.8345) tensor(0.1655)
tensor(0.7235) tensor(0.2765)
tensor(0.6455) tensor(0.3545)
tensor(0.8330) tensor(0.1670)
tensor(0.8550) tensor(0.1450)
tensor(0.8119) tensor(0.1881)
tensor(0.8020) tensor(0.1980)
tensor(0.6007) tensor(0.3993)
tensor(0.6322) tensor(0.3678)
tensor(0.8738) tensor(0.1262)
tensor(0.8395) tensor(0.1605)
tensor(0.8376) tensor(0.1624)
tensor(0.6577) tensor(0.3423)
tensor(0.7986) tensor(0.2014)
tensor(0.8465) tensor(0.1535)
tensor(0.8718) tensor(0.1282)
tensor(0.8637) tensor(0.1363)
tensor(0.6550) tensor(0.3450)
tensor(0.6355) tensor(0.3645)
tensor(0.7898) tensor(0.2102)
tensor(0.7168) tensor(0.2832)
tensor(0.7343) tensor(0.2657)
tensor(0.6257) tensor(0.3743)
tensor(0.7907) tensor(0.2093)
tensor(0.8236) tensor(0.1764)
tensor(0.8

In [26]:
# import torch

# promptModel.eval()
# with torch.no_grad():
#     for batch in data_loader:
#         logits = promptModel(batch)
#         print(batch.tgt_text)
#         preds = torch.argmax(logits, dim = -1)
#         #print(preds)
#         print(classes[preds])


In [27]:
query_dict = dict(sorted_queries)

In [28]:
query_dict = {k: v.item() for k, v in query_dict.items()}

In [29]:
query_dict

{'405717': 33.236907958984375,
 '1103812': 32.94999694824219,
 '1063750': 32.426876068115234,
 '1121402': 28.943124771118164,
 '1106007': 28.62481689453125,
 '1113437': 28.548147201538086,
 '1112341': 27.851943969726562,
 '1117099': 26.81060028076172,
 '168216': 25.327800750732422,
 '962179': 24.187931060791016,
 '1124210': 23.475482940673828,
 '1133167': 22.292003631591797,
 '1115776': 21.909236907958984,
 '1114646': 20.34986114501953,
 '87181': 19.968477249145508,
 '183378': 19.962139129638672,
 '47923': 19.40315055847168,
 '915593': 18.749189376831055,
 '855410': 18.594324111938477,
 '833860': 17.497726440429688,
 '19335': 16.73459243774414,
 '156493': 15.948209762573242,
 '182539': 14.876989364624023,
 '451602': 14.73635482788086,
 '490595': 13.301484107971191,
 '1037798': 12.844978332519531,
 '87452': 11.711054801940918,
 '527433': 10.621183395385742,
 '148538': 9.990985870361328,
 '146187': 9.50208568572998,
 '359349': 9.442964553833008,
 '443396': 9.251060485839844,
 '207786': 7

In [30]:
with open('trecdl19rel.txt', 'r') as f:
    gt = dict(line.strip().split() for line in f)

with open('bertpred.txt', 'r') as f:
    bertpred = dict(line.strip().split() for line in f)

query_scores = list(query_dict.values())
gtscores = [int(gt[key]) for key in query_dict.keys()]
bertscores = [int(bertpred[key]) for key in query_dict.keys()]


In [31]:
l = list(query_dict.keys())

In [32]:
with open('gpt2(large)1.txt', 'w') as f:
    for key, score in zip(l, query_scores):
        f.write(f'{key}\t{score}\n')

In [33]:
# import numpy as np
# pccs = np.corrcoef(scores, gtscores)

In [34]:
import numpy as np
def cal_pccs(X, Y):
    XMean = np.mean(X)
    YMean = np.mean(Y)
    #标准差
    XSD = np.std(X)
    YSD = np.std(Y)
    #z分数
    ZX = (X-XMean)/XSD
    ZY = (Y-YMean)/YSD#相关系数
    r = np.sum(ZX*ZY)/(len(X))
    return(r)


In [37]:
pc1 = cal_pccs(query_scores, gtscores)
pc2 = cal_pccs(bertscores, gtscores)
print(pc1)
print(pc2)

0.14288053980928345
0.20646485309897292


In [38]:
from scipy.stats.stats import kendalltau
k1 = kendalltau(query_scores, gtscores)
k2 = kendalltau(bertscores, gtscores)
print(k1)
print(k2)

KendalltauResult(correlation=0.0942959482520245, pvalue=0.3799248289061834)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)


In [12]:
# from gensim.summarization import bm25
# import jieba
gpt2:
0.06533198011630328
0.20646485309897283
KendalltauResult(correlation=0.06635640802920242, pvalue=0.5366568517716708)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)

albert:
0.10515481453939302
0.20646485309897283
KendalltauResult(correlation=0.07334129308490794, pvalue=0.49465994677884506)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)

roberta-large:
0.12948093733527669
0.2064648530989729
KendalltauResult(correlation=0.0942959482520245, pvalue=0.3799248289061834)
KendalltauResult(correlation=0.16931033807650897, pvalue=0.13238651626659328)




# def test_gensim_bm25():
#     # 给定多个文档
#     corpus = ["5万元资金，该做什么行业",
#               "美增加汽车关税，为何汽车价格不降反升",
#               "汽车销售人员的服务水准非常烂，该怎么解决",
#               "未来房价会跌到什么程度",
#               "十万元能上路的汽车，买什么比较好"]
#     # 对每个文档切词（示例作用  不进行去停用词）
#     corpus_cut = [jieba.lcut(line) for line in corpus]

#     # 生成模型
#     bm25Model = bm25.BM25(corpus_cut)

#     test_query = "你想买汽车吗"  # query
#     test_query_cut = jieba.lcut(test_query)

#     scores = bm25Model.get_scores(test_query_cut)  # 计算相似度得分(与corpus_cut顺序对应)
#     print("scores", scores)
#     # 输出
#     for i, j in zip(scores, corpus):
#         print('分值：{},原句：{}'.format(i, j))
#     print('\n')

